In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, models
from torchvision.utils import make_grid


import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import os
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [0]:
class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.model = nn.Sequential(
            self.conv_bn(  3,  32, 2), 
            self.conv_dw( 32,  64, 1),
            self.conv_dw( 64, 128, 2),
            self.conv_dw(128, 128, 1),
            self.conv_dw(128, 256, 2),
            self.conv_dw(256, 256, 1),
            self.conv_dw(256, 512, 2),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 512, 1),
            self.conv_dw(512, 1024, 2),
            self.conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7)
            )
        self.fc = nn.Linear(1024, 1000)
        
        
    def conv_bn(self, inp, oup, stride):
      return nn.Sequential(
          nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
          nn.BatchNorm2d(oup),
          nn.ReLU(inplace=True)
          )
      
    def conv_dw(self, inp, oup, stride):
      return nn.Sequential(
          nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
          nn.BatchNorm2d(inp),
          nn.ReLU(inplace=True),
          nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
          nn.BatchNorm2d(oup),
          nn.ReLU(inplace=True)
          )

    
    def forward(self,X):
      X = self.model(X)
      X = X.view(-1, 1024)
      X = self.fc(X)
      return F.log_softmax(X, dim=1)

In [76]:
MobileNet()

MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 128, kernel_size=(1, 1),

In [0]:
x = torch.randn(1,3,224,224)

In [78]:
model(x).shape

torch.Size([1, 1000])